In [1]:
import pandas as pd
import numpy as np
import os
import sys
import requests
import re
from datetime import datetime

In [2]:
pd.options.display.max_columns = 150
pd.options.display.max_rows = 150

In [3]:
class Preprocesser:
    def __init__(self):
        self.kospi = ""
        self.kosdaq = ""
        # need kospi file
        # MK Embedding file
    
        #papago 번역 함수
    
    
    #폴더에 있는 파일 리스트 가져오기
    def get_filelist(self):
        filelist = os.listdir(os.getcwd())
        return filelist


    #파일 불러오기
    def load_funda_data(self, data_name):
        data = pd.read_excel("{}".format(data_name),encoding='cp949', thousands=',')
        return data    

    def load_feature_data(self, data_name):
        if 'csv' in data_name:
            data = pd.read_csv("{}".format(data_name), thousands=',')
        else:
            data = pd.read_excel("{}".format(data_name),encoding='cp949', thousands=',')


    # 재무 데이터 컬럼 이름 설정
    def set_col_names(self, df):
        col_list = ['Date', 'asset(1000)', 'debt(1000)', 'sales(1000)',
                    'adjusted_price', 'profit_rate', 'volumne(mean_5d)(week)',
                    'volumne(week)','profit(1000)',
                    'term_profit(1000)', 'gross_margin(1000)','adjusted_BPS', 'adjusted_EPS']
        df.columns = col_list
        return df
    
    def add_feature_name(self, feature_name_list, file_name):
        dotpos = file_name.find('.')
        file_name = file_name[:dotpos]
        new_col = ['Date']
        for i in range(feature_name_list)-1:
            new_name = feature_name_list[i+1]+file_name
            new_col += new_name
        return new_col


#     # 피쳐 데이터 컬럼 이름 설정(2차전지) (나머지 산업에 대해서도 필요)
#     def feature_col_name(df, name):
#         """
#         name : dubai, nickel, mirenal, wti, lithum, cobalt_l,cobalt_r, brent

#         """
#         if name =='dubai':
#             col_list = dubai_price_col_list = ['Date', 'dubai_price','dubai_price_change','dubai_price_change_rate','dubai_stock']
#         elif name == 'nickel':
#             col_list = nickel_price_col_list = ['Date', 'nickel_price','nickel_price_change','nickel_price_change_rate','nikcel_stock']
#         elif name == 'mineral':
#             col_list = mineral_col_list = ['Date', 'mineral_index','mineral_index_change','mineral_index_change_rate']
#         elif name == 'wti': 
#             col_list = wti_price_col_list = ['Date', 'wti_price','wti_price_change','wti_price_change_rate','wti_stock']
#         elif name == 'lithum':
#             col_list = lithum_price_col_list = ['Date', 'lithum_price','lithum_price_change','lithum_price_change_rate','lithum_stock']
#         elif name == 'cobalt_l': 
#             col_list = cobalt_l_price_col_list = ['Date', 'cobalt_l_price','coblat_l_price_change','cobalt_l_price_change_rate','cobalt_l_stock']
#         elif name == 'cobalt_r': 
#             col_list = cobalt_r_price_col_list = ['Date', 'cobalt_r_price','coblat_r_price_change','cobalt_r_price_change_rate','cobalt_r_stock']
#         elif name == 'brent': 
#             col_list = brent_price_col_list = ['Date', 'brent_price','brent_price_change','brent_price_change_rate','brent_stock']
        
#         df.columns = col_list

#         return df


    # 인덱스 설정하고 원하는 연도부터 프레임 설정(재무)
    def set_funda_data(self, data, start_date):

        """
        set_funda_data(df,'2010-01-05')
        """
        data = data.iloc[5:]

        data.Date =  pd.to_datetime(data.Date)
        data.set_index(['Date'],inplace=True)
        data = data.loc[start_date:]
        return data


    # 인덱스 설정하고 원하는 연도부터 프레임 설정(피쳐)
    def set_feature_data(self, data, start_date):

        """
        set_funda_data('samsung_elec",'2010-01-05')
        """
        
        data.Date =  pd.to_datetime(data.Date)
        data.set_index(['Date'],inplace=True)
        data = data.loc[start_date:]
        return data



    #화장품 fundamental data 내 존재하는 monthly data를 날짜별로 만들기  ( 현재 datetime으로 되어 있고, 모든 날짜에 값 부여 필요 )
    def mkdatetime_mth(self, data):
        data2 = data[data['년월']['년월'].str.contains("월")]
        index = data2.index
        for i, mth in enumerate(data2['년월']['년월']):
            m = int(mth[1:-1])
            idx = index[i]
            idx = idx - m
            y = int(data['년월']['년월'][idx][:-1])
            print(y, m)

            data2.iat[i, 0] = datetime(y, m, 1)

        return data2






    # #데이터 불러와서 datetime으로 만들고, 각 컬럼 모두 합치기, 수연이한테 각 컬럼명이 무엇을 뜻하는 지 물어볼 것
    # def read_mth_data(self, data, domain):
    #     domain = "화장품"
    #     os.chdir(r"G:\공유 드라이브\Boad ADV Stock\{}\fundamental".format(domain))
        
    #     data = pd.read_excel("330112_오렌지유.xls", skiprows = [0, 1], header=[0,1])
    #     pd.read_excel("330113_레몬유.xls", skiprows = [0, 1], header=[0,1])



    # 특정 연도 사이 휴일 리스트
    def check_holiday(self, start_year, last_year):
        
        """
        check_hodiday(2010,2020)
        """

        total_holiday = list()
        for year in range(start_year, last_year):
            hlist = list()
            krholiday = [d.strftime('%Y-%m-%d') for d in pytimekr.holidays(year)]
            week_sat = [d.strftime('%Y-%m-%d') for d in pd.date_range('{}-01-01'.format(year),'{}-12-31'.format(year),freq='W-SAT')]
            week_sun = [d.strftime('%Y-%m-%d') for d in pd.date_range('{}-01-01'.format(year),'{}-12-31'.format(year),freq='W-SUN')]
            hlist.extend(krholiday)
            hlist.extend(week_sat)
            hlist.extend(week_sun)
            total_holiday.extend(pd.unique(hlist))
        sorted(total_holiday)
        return total_holiday


    # 특정 연도 사이 휴일 x 리스트
    def check_workday(self, start_year, last_year):
        """
        check_workday(2010,2020)
        """

        workday = list()
        for d in pd.date_range('{}-01-01'.format(start_year),'{}-12-31'.format(last_year)):
            if d.strftime('%Y-%m-%d') not in check_holiday(start_year, last_year):
                workday.append(d.strftime('%Y-%m-%d'))       
        return workday


    # 데이터 프레임에서 휴일 아닌 날만 뽑기
    def set_index_workday(self, df,start_year,last_year):
        work = pd.DataFrame(check_workday(start_year, last_year),columns=['Date'])
        work.Date =  pd.to_datetime(work.Date)
        df.reset_index(inplace=True)
        work_df = pd.merge(df,work, on ='Date',how='inner')
        work_df.set_index(['Date'],inplace=True)
        # work_df = pd.concat([df,work], join_axes=['Date'],join='inner',axis=0)

        return work_df

    # 회시별 재무 데이터 테이블
    def individual_feauture(self, stockname):
        df = load_funda_data(stockname)
        df = set_col_names(df)
        df = set_funda_data(df)
        return df

    # 산업 공통 피쳐
    def common_feature(self,featurename):
        
        df = load_funda_data(featurename)
        df = feature_col_name(df)
        df = set_feature_data(df)
        return df

    # 산업 공통 피쳐 테이블
    def common_feature_table(self):
        os.chdir(r'G:\공유 드라이브\Boad ADV Stock\2차전지\funda_data')
        filelist = os.listdir(os.getcwd())
        # filelist
        feature_df = pd.DataFrame()
        for feature in filelist:
            add_df = common_feature(feature)
            feature_df = pd.merge(feature_df, add_df, on='Date', how ='left')
        return


In [4]:
class cosmetic_preprocesser(Preprocesser):
    
    def get_filelist(self, folder):
        #folder = 'fn_data' , 'tech_data'
        filelist = os.listdir(os.chdir(r'G:\공유 드라이브\Boad ADV Stock\{}\{}'.format(industry,folder)))
        return filelist

In [5]:
os.chdir(r'G:\공유 드라이브\Boad ADV Stock\화장품')

In [6]:
os.getcwd()

'G:\\공유 드라이브\\Boad ADV Stock\\화장품'

In [7]:
os.chdir('./fn_data')

In [8]:
cosmetic_preprocesser = cosmetic_preprocesser()

In [9]:
industry = '화장품'
cosmetic_preprocesser.get_filelist('fn_data')

['아모레퍼시픽.xlsx',
 '코리아나.xlsx',
 '코스맥스.xlsx',
 '한국콜마.xlsx',
 '한국화장품제조.xlsx',
 '한국화장품.xlsx']

In [10]:
# cosmetic_preprocesser.get_filelist('fn_data')

In [11]:
data  = cosmetic_preprocesser.load_funda_data(cosmetic_preprocesser.get_filelist('fn_data')[1])

In [12]:
company = '한국화장품'
data = pd.read_excel('{}.xlsx'.format(company))

In [13]:
data = cosmetic_preprocesser.set_col_names(data)
data.head(10)

,Date,asset(1000),debt(1000),sales(1000),adjusted_price,profit_rate,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),adjusted_BPS,adjusted_EPS
0,Symbol Name,한국화장품,한국화장품,한국화장품,한국화장품,한국화장품,한국화장품,한국화장품,한국화장품,한국화장품,한국화장품,한국화장품,한국화장품
1,Kind,NFS-IFRS(C),NFS-IFRS(C),NFS-IFRS(C),SSC,SSC,SSC,SSC,NFS-IFRS(C),NFS-IFRS(C),NFS-IFRS(C),NVAL-IFRS(C),NVAL-IFRS(C)
2,Item,6000901001,6000902001,6000904001,S410000700,S41000170F,S41000620F,S41000080F,6000906001,6000908001,6000904007,6000601036,6000601003
3,Item Name,총자산(천원),총부채(천원),매출액(천원),수정주가(원),수익률(%),거래량 (5일 평균)(주),거래량(주),영업이익(천원),당기순이익(천원),매출총이익(천원),수정BPS(원),수정EPS(원)
4,Frequency,1Q,1Q,1Q,DAILY,DAILY,DAILY,DAILY,1Q,1Q,1Q,1Q,1Q
5,2009-01-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2009-01-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2009-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2009-01-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2009-01-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
data = cosmetic_preprocesser.set_funda_data(data , '2010-01-11')
data.head()

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,asset(1000),debt(1000),sales(1000),adjusted_price,profit_rate,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),adjusted_BPS,adjusted_EPS
Date,,,,,,,,,,,,
2010-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
data = data.reset_index()
data.replace('-',np.nan, inplace=True)
data.head()

,Date,asset(1000),debt(1000),sales(1000),adjusted_price,profit_rate,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),adjusted_BPS,adjusted_EPS
0,2010-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
data.Date = pd.to_datetime(data.Date)

In [17]:
data.head()

,Date,asset(1000),debt(1000),sales(1000),adjusted_price,profit_rate,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),adjusted_BPS,adjusted_EPS
0,2010-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# feature data

In [18]:
cosmetic_preprocesser.get_filelist('feature_data')

['kospi_volatility.csv',
 'South Korea 5-Year Bond Yield Historical Data.csv',
 'USD_KRW 내역.csv',
 '외국인입국통계__202001.xlsx',
 'kospi,kosdaq.xlsx',
 'CNY_KRW 내역.csv']

In [19]:
def add_feature_name(feature_name_list, file_name):
    dotpos = file_name.find('.')
    name = file_name
    new_col = ['Date']
    for i in feature_name_list[1:]:
        new_name = i + '_' +name
        new_col.append(new_name)
    return new_col

# kospi volatility

In [20]:
kospi_vol_name = cosmetic_preprocesser.get_filelist('feature_data')[0]
kospi_vol_name

'kospi_volatility.csv'

In [21]:
kospi_vol = pd.read_csv("{}".format(kospi_vol_name), thousands=',')
kospi_vol.head()

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2019년 12월 30일,14.69,14.86,15.00,14.69,-,3.96%
1,2019년 12월 27일,14.13,14.36,14.72,14.13,-,-0.14%
2,2019년 12월 26일,14.15,14.11,14.20,14.03,-,2.46%
3,2019년 12월 24일,13.81,14.16,14.23,13.81,-,-1.15%
4,2019년 12월 23일,13.97,13.57,14.05,13.57,-,6.80%


In [22]:
kospi_vol.columns = ['Date','last','open','high','low','volume','%change']

kospi_vol.columns = add_feature_name(kospi_vol.columns,'kospi_vol')
# kospi_vol = kospi_vol.iloc[:,:4]
# cobalt_LME= co_lithum[['Date', 'price_co_lithum', 'change_co_lithum', '%change_co_lithum']]

kospi_vol.head()

,Date,last_kospi_vol,open_kospi_vol,high_kospi_vol,low_kospi_vol,volume_kospi_vol,%change_kospi_vol
0,2019년 12월 30일,14.69,14.86,15.00,14.69,-,3.96%
1,2019년 12월 27일,14.13,14.36,14.72,14.13,-,-0.14%
2,2019년 12월 26일,14.15,14.11,14.20,14.03,-,2.46%
3,2019년 12월 24일,13.81,14.16,14.23,13.81,-,-1.15%
4,2019년 12월 23일,13.97,13.57,14.05,13.57,-,6.80%


In [23]:
kospi_vol = kospi_vol[['Date', 'last_kospi_vol', 'open_kospi_vol', 'high_kospi_vol',
       'low_kospi_vol', '%change_kospi_vol']]

In [24]:
kospi_vol['%change_kospi_vol'] = kospi_vol['%change_kospi_vol'].str.replace('%','')
kospi_vol.Date = kospi_vol.Date.str.replace('년 ','')
kospi_vol.Date = kospi_vol.Date.str.replace('월 ','')
kospi_vol.Date = kospi_vol.Date.str.replace('일','')
kospi_vol.Date = pd.to_datetime(kospi_vol.Date, format='%Y %m %d')
kospi_vol.sort_values(by='Date', inplace=True)
kospi_vol.head()

,Date,last_kospi_vol,open_kospi_vol,high_kospi_vol,low_kospi_vol,%change_kospi_vol
1569,2013-08-06,14.98,14.79,14.98,14.61,0.40
1568,2013-08-07,15.35,15.02,15.42,14.80,2.47
1567,2013-08-08,14.96,14.84,15.42,14.77,-2.54
1566,2013-08-09,14.38,14.95,15.05,14.38,-3.88
1565,2013-08-12,14.72,14.84,15.22,14.64,2.36


# 오렌지유

In [25]:
# orange_name = cosmetic_preprocesser.get_filelist('feature_data')[1]
# orange_name

In [26]:
# orange_oil = pd.read_excel("{}".format(orange_name), thousands=',')
# orange_oil.head()

In [27]:
# orange_oil.head(10)

In [28]:
# orange_oil.columns = ['Date','ex_price','ex_pr_change','ex_weight','ex_we_change','im_price','im_pr_change','im_weight','im_we_change']

# orange_oil.columns = add_feature_name(orange_oil.columns,'orange_oil')
# # kospi_vol = kospi_vol.iloc[:,:4]
# # cobalt_LME= co_lithum[['Date', 'price_co_lithum', 'change_co_lithum', '%change_co_lithum']]

# orange_oil.head()

In [29]:
# orange_oil.iloc[4:,:]

# korean bond

In [30]:
korean_bond_name = cosmetic_preprocesser.get_filelist('feature_data')[1]
korean_bond_name

'South Korea 5-Year Bond Yield Historical Data.csv'

In [31]:
korean_bond = pd.read_csv("{}".format(korean_bond_name), thousands=',')
korean_bond.head()

,Date,Price,Open,High,Low,Change %
0,"Dec 31, 2019",1.470,1.470,1.471,1.470,-0.74%
1,"Dec 30, 2019",1.481,1.472,1.487,1.472,-0.27%
2,"Dec 29, 2019",1.485,1.485,1.485,1.485,0.00%
3,"Dec 28, 2019",1.485,1.485,1.485,1.485,0.00%
4,"Dec 27, 2019",1.485,1.496,1.496,1.480,1.02%


In [32]:
korean_bond.columns = ['Date','last','open','high','low','%change']

korean_bond.columns = add_feature_name(korean_bond.columns,'korean_bond')
# usd_krw = usd_krw.iloc[:,:4]
# cobalt_LME= co_lithum[['Date', 'price_co_lithum', 'change_co_lithum', '%change_co_lithum']]

korean_bond.head()

,Date,last_korean_bond,open_korean_bond,high_korean_bond,low_korean_bond,%change_korean_bond
0,"Dec 31, 2019",1.470,1.470,1.471,1.470,-0.74%
1,"Dec 30, 2019",1.481,1.472,1.487,1.472,-0.27%
2,"Dec 29, 2019",1.485,1.485,1.485,1.485,0.00%
3,"Dec 28, 2019",1.485,1.485,1.485,1.485,0.00%
4,"Dec 27, 2019",1.485,1.496,1.496,1.480,1.02%


In [33]:
korean_bond['%change_korean_bond'] = korean_bond['%change_korean_bond'].str.replace('%','')
korean_bond.Date = korean_bond.Date.str.replace(',','')
korean_bond.Date = pd.to_datetime(korean_bond.Date, format='%b %d %Y')
korean_bond.sort_values(by='Date', inplace=True)
korean_bond.head()

,Date,last_korean_bond,open_korean_bond,high_korean_bond,low_korean_bond,%change_korean_bond
2977,2010-01-04,4.960,4.960,4.960,4.960,1.12
2976,2010-01-05,4.949,4.949,4.949,4.949,-0.22
2975,2010-01-06,4.880,4.880,4.880,4.880,-1.39
2974,2010-01-07,4.920,4.920,4.920,4.920,0.82
2973,2010-01-08,4.910,4.910,4.910,4.910,-0.20


# usd_krw

In [34]:
usd_krw_name = cosmetic_preprocesser.get_filelist('feature_data')[2]
usd_krw_name

'USD_KRW 내역.csv'

In [35]:
usd_krw = pd.read_csv("{}".format(usd_krw_name), thousands=',')
usd_krw.head()

,날짜,종가,오픈,고가,저가,변동 %
0,2019년 12월 31일,1155.07,1157.97,1159.19,1153.14,-0.25%
1,2019년 12월 30일,1157.97,1160.97,1161.30,1154.89,-0.15%
2,2019년 12월 27일,1159.68,1162.15,1162.83,1158.12,-0.23%
3,2019년 12월 26일,1162.30,1161.04,1163.42,1160.34,0.13%
4,2019년 12월 25일,1160.76,1163.52,1164.09,1160.68,-0.24%


In [36]:
usd_krw.columns = ['Date','last','open','high','low','%change']
usd_krw.sort_values(by='Date', inplace=True)
usd_krw.columns = add_feature_name(usd_krw.columns,'usd_krw')
# usd_krw = usd_krw.iloc[:,:4]
# cobalt_LME= co_lithum[['Date', 'price_co_lithum', 'change_co_lithum', '%change_co_lithum']]

usd_krw.head()

,Date,last_usd_krw,open_usd_krw,high_usd_krw,low_usd_krw,%change_usd_krw
2607,2010년 01월 01일,1166.08,1166.08,1166.08,1166.08,0.00%
2606,2010년 01월 04일,1150.00,1164.07,1164.07,1149.35,-1.38%
2605,2010년 01월 05일,1140.20,1151.00,1151.00,1136.30,-0.85%
2604,2010년 01월 06일,1129.20,1140.20,1144.70,1129.20,-0.96%
2603,2010년 01월 07일,1138.20,1129.20,1138.20,1127.80,0.80%


In [37]:
usd_krw['%change_usd_krw'] = usd_krw['%change_usd_krw'].str.replace('%','')
usd_krw.Date = usd_krw.Date.str.replace('년 ','')
usd_krw.Date = usd_krw.Date.str.replace('월 ','')
usd_krw.Date = usd_krw.Date.str.replace('일','')
usd_krw.Date = pd.to_datetime(usd_krw.Date, format='%Y %m %d')
usd_krw.head()

,Date,last_usd_krw,open_usd_krw,high_usd_krw,low_usd_krw,%change_usd_krw
2607,2010-01-01,1166.08,1166.08,1166.08,1166.08,0.00
2606,2010-01-04,1150.00,1164.07,1164.07,1149.35,-1.38
2605,2010-01-05,1140.20,1151.00,1151.00,1136.30,-0.85
2604,2010-01-06,1129.20,1140.20,1144.70,1129.20,-0.96
2603,2010-01-07,1138.20,1129.20,1138.20,1127.80,0.80


# Foreign

In [38]:
foreigner_name = cosmetic_preprocesser.get_filelist('feature_data')[3]
foreigner_name

'외국인입국통계__202001.xlsx'

In [39]:
foreigner = pd.read_excel("{}".format(foreigner_name), thousands=',')
foreigner = foreigner.T
foreigner.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,...,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289
［ 입 국 ］,NaN,Country,G.TOTAL,NaN,Foreign Visitors,Overseas Korean,NaN,ASIA,NaN,Japan,Taiwan,"Hong Kong, China(including Hong Kong ID.)",Macao,Thailand,Malaysia,Philippines,Indonesia,Singapore,Brunei,Myanmar,Laos,Vietnam,East-Timor,India,Sri Lanka,Pakistan,Nepal,Bangladesh,Afghanistan,Cambodia,Bhutan,Maldives,Mongolia,Sikkim,China P. R.,(China),(China-Korean),D.P.R Korea,Mayotte,British Indian Ocean Territory,United Nations,others,NaN,Middle East Asia,NaN,Iran,Saudi Arabia,Israel,Jordan,Lebanon,Kuwait,Syria,Turkey,Yemen,Yemen P.D.R.,Qatar,U.A.E,Oman,Iraq,Bahrain,Cyprus,Palestine,Uzbekistan,Kazakhstan,Kyrgyzstan,Turkmenistan,Tadjikistan,others,NaN,Americas,NaN,United States,Canada,Mexico,Nicaragua,...,Christmas I.,Wake I.,Midway Is.,Cocos(Keeling) I.,Niue,Pitcairn,Tokelau,others,NaN,AFRICA,NaN,Nigeria,Egypt,South Sudan,South Africa,Tunisia,Libya,Ghana,Morocco,Sudan,Ethiopia,Mauritius,Zambia,Uganda,Tanzania,Kenya,Liberia,Gabon,Mali,Somalia,Algeria,Guinea Bissau,Malawi,Ivory Coast,Senegal,Zaire,Zimbabwe,Cameroon,Swaziland,Upper Volta,Congo P.R.,Democratic Republic Of The Congo,Madagascar,Malagasy,Burundi,Mauritania,Lesotho,Sierra Leone,Guinea,Niger,Angola,Chad,Equator Guinea,Central Africa,Djibouti,Seychelles,Botswana,Namibia,Cape Verde,Gambia,Togo,Canary Is.,Rwanda,Comoros,Saotome-Principe,Mozambique,Benin,Western Sahara,Burkina Faso,St.Helena,Reunion,Eritrea,others,NaN,Stateless
Unnamed: 1,NaN,국가명,NaN,NaN,NaN,NaN,NaN,NaN,NaN,일본,대만,홍콩,마카오,태국,말레이시아,필리핀,인도네시아,싱가포르,브루나이,미얀마,라오스,베트남,티모르,인도,스리랑카,파키스탄,네팔,방글라데시,아프가니스탄,캄보디아,부탄,몰디브,몽골,시킴,중국,(중국1),(한국계중국인),북한,마요테,영령인도양섬,국제연합,(아시아기타),NaN,중동,GCC6개국,이란,사우디아라비아,이스라엘,요르단,레바논,쿠웨이트,시리아,터키,예멘공화국,예멘인민민주주의공화국,카타르,아랍에미리트연합,오만,이라크,바레인,키프로스,팔레스타인,우즈베키스탄,카자흐스탄,키르기스스탄,투르크메니스탄,타지키스탄,중동 기타,NaN,미주,NaN,미국,캐나다,멕시코,니카라과,...,크리스마스,웨이크아일랜드,미드웨이군도,코코스군도,니우에,핏캐른,토켈라우,대양주 기타,NaN,아프리카,NaN,나이지리아,이집트,남수단공화국,남아프리카 공화국,튀니지,리비아,가나,모로코,수단,에티오피아,모리셔스,잠비아,우간다,탄자니아,케냐,라이베리아,가봉,말리,소말리아,알제리,기니비사우,말라위,코트디부아르,세네갈,자이르,짐바브웨,카메룬,스와질랜드,오트볼타,콩고,콩고 민주공화국,마다가스카르,말라가시,부룬디,모리타니,레소토,시에라리온,기니,니제르,앙골라,차드,적도기니,중앙 아프리카 공화국,지부티,세이셸,보츠와나,나미비아,카보베르데,감비아,토고,카나리아 군도,르완다,코모로,상투메 프린시페,모잠비크,베냉,서사하라,부르키나파소,세인트 헬레나,레위니옹,에리트리아,아프리카 기타,NaN,무국적*
Unnamed: 2,NaN,1984년,1297318,NaN,1124076,173242,NaN,802601,NaN,576448,93543,46587,15,10107,25634,8049,9272,12727,310,126,0,5,0,5845,1650,1620,527,352,6,2,7,85,0,1,108,108,0,0,0,0,0,0,NaN,9575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,231662,NaN,212986,13192,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 3,NaN,1985년,1426045,NaN,1253228,172817,NaN,883308,NaN,638941,99622,47110,20,10527,30307,10350,10753,12980,408,163,3,23,0,6279,1704,2666,348,673,16,0,27,60,4,0,308,308,0,0,0,0,0,0,NaN,10016,4031,2544,2536,1074,450,421,647,215,567,167,40,151,209,113,467,375,38,2,0,0,0,0,0,0,NaN,263504,NaN,239423,16964,1052,26,...,0,0,0,0,0,0,0,0,NaN,5323,NaN,517,1006,NaN,414,197,825,128,126,131,40,75,19,35,29,112,42,92,19,28,27,28,20,57,81,54,60,53,4,14,8,0,15,0,5,39,8,22,23,9,0,7,10,13,2,4,12,0,11,22,13,0,6,9,4,4,21,0,1,0,0,0,0,NaN,822
Unnamed: 4,NaN,1986년,1659972,NaN,1457711,202261,NaN,1032985,NaN,791011,94799,55315,23,11313,16674,14764,9977,10973,215,250,0,154,0,7443,799,4060,487,1118,20,2,46,49,

In [40]:
foreigner.columns = foreigner.iloc[0,:]

In [41]:
foreigner = foreigner.iloc[173:,:]

In [42]:
foreigner.head()

［ 입 국 ］,nan,Country,G.TOTAL,nan,Foreign Visitors,Overseas Korean,nan,ASIA,nan,Japan,Taiwan,"Hong Kong, China(including Hong Kong ID.)",Macao,Thailand,Malaysia,Philippines,Indonesia,Singapore,Brunei,Myanmar,Laos,Vietnam,East-Timor,India,Sri Lanka,Pakistan,Nepal,Bangladesh,Afghanistan,Cambodia,Bhutan,Maldives,Mongolia,Sikkim,China P. R.,(China),(China-Korean),D.P.R Korea,Mayotte,British Indian Ocean Territory,United Nations,others,nan,Middle East Asia,nan,Iran,Saudi Arabia,Israel,Jordan,Lebanon,Kuwait,Syria,Turkey,Yemen,Yemen P.D.R.,Qatar,U.A.E,Oman,Iraq,Bahrain,Cyprus,Palestine,Uzbekistan,Kazakhstan,Kyrgyzstan,Turkmenistan,Tadjikistan,others,nan,Americas,nan,United States,Canada,Mexico,Nicaragua,...,Christmas I.,Wake I.,Midway Is.,Cocos(Keeling) I.,Niue,Pitcairn,Tokelau,others,nan,AFRICA,nan,Nigeria,Egypt,South Sudan,South Africa,Tunisia,Libya,Ghana,Morocco,Sudan,Ethiopia,Mauritius,Zambia,Uganda,Tanzania,Kenya,Liberia,Gabon,Mali,Somalia,Algeria,Guinea Bissau,Malawi,Ivory Coast,Senegal,Zaire,Zimbabwe,Cameroon,Swaziland,Upper Volta,Congo P.R.,Democratic Republic Of The Congo,Madagascar,Malagasy,Burundi,Mauritania,Lesotho,Sierra Leone,Guinea,Niger,Angola,Chad,Equator Guinea,Central Africa,Djibouti,Seychelles,Botswana,Namibia,Cape Verde,Gambia,Togo,Canary Is.,Rwanda,Comoros,Saotome-Principe,Mozambique,Benin,Western Sahara,Burkina Faso,St.Helena,Reunion,Eritrea,others,nan,Stateless
Unnamed: 173,NaN,2010.1월,569453,NaN,546916,22537,NaN,433045,NaN,209184,36399,14545,554,17814,4559,21214,5061,5368,27,4594,92,4515,41,6026,648,714,519,710,29,280,11,14,3324,NaN,91252,77291,13961,NaN,NaN,NaN,18,NaN,NaN,5533,430,598,220,856,231,95,32,130,969,31,NaN,6,130,37,196,5,22,15,1280,419,204,7,50,NaN,NaN,53839,NaN,43065,8776,379,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2138,NaN,176,330,NaN,788,56,125,130,38,76,20,24,4,18,122,34,4,5,3,NaN,64,NaN,1,12,4,NaN,2,16,1,NaN,1,12,2,NaN,2,1,NaN,10,4,NaN,13,1,NaN,NaN,NaN,8,NaN,16,1,3,4,NaN,2,NaN,NaN,1,2,NaN,2,NaN,NaN,NaN,NaN,NaN,13
Unnamed: 174,NaN,2010.2월,638911,NaN,617573,21338,NaN,515506,NaN,230362,36651,23051,918,16683,7483,20411,4922,5798,40,4464,78,6749,7,5439,696,786,481,659,37,399,6,3,3582,NaN,140571,117125,23446,NaN,NaN,NaN,50,0,NaN,5180,472,491,247,844,198,71,48,97,874,37,NaN,11,96,50,83,20,21,11,1296,469,163,8,45,NaN,NaN,51756,NaN,41077,8503,429,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2350,NaN,159,291,NaN,876,36,150,145,51,83,41,15,6,10,176,70,4,3,NaN,1,56,NaN,NaN,8,16,NaN,9,32,2,NaN,5,8,5,NaN,1,NaN,1,2,2,4,38,NaN,NaN,NaN,1,9,NaN,10,3,NaN,3,NaN,9,NaN,NaN,3,4,NaN,2,NaN,NaN,NaN,NaN,NaN,10
Unnamed: 175,NaN,2010.3월,769894,NaN,742113,27781,NaN,602029,NaN,306126,27894,17924,435,25158,8333,27441,6030,7409,95,5412,136,7674,10,6810,684,852,475,632,54,394,18,60,3293,NaN,141457,102246,39211,NaN,NaN,NaN,35,NaN,NaN,7188,542,818,281,1239,270,100,55,141,1492,52,NaN,11,141,32,163,22,28,24,1510,510,250,8,41,NaN,NaN,67540,NaN,54610,10001,530,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3005,NaN,255,597,NaN,688,58,170,206,89,134,41,65,10,18,224,81,12,8,2,1,41,2,1,13,13,NaN,8,41,2,NaN,6,77,4,NaN,3,3,NaN,1,4,3,74,NaN,NaN,NaN,NaN,7,2,13,1,2,1,NaN,11,NaN,NaN,2,9,NaN,NaN,NaN,NaN,2,NaN,NaN,21
Unnamed: 176,NaN,2010.4월,730265,NaN,702568,27697,NaN,561228,NaN,239075,33518,20217,473,33398,11591,26529,7516,8732,101,5068,171,7152,11,7155,559,675,561,577,49,547,11,17,2512,NaN,147680,125230,22450,NaN,NaN,NaN,45,NaN,NaN,7288,691,614,304,1165,271,108,95,176,1676,53,NaN,20,200,51,196,21,29,19,1296,804,152,5,33,NaN,NaN,73194,NaN,59616,10708,536,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2561,NaN,269,355,NaN,669,73,128,179,85,82,71,58,5,7,192,70,7,12,3,NaN,77,NaN,9,12,14,NaN,27,29,NaN,NaN,2,31,14,NaN,NaN,1,NaN,1,6,8,19,1,NaN,NaN,NaN,4,2,17,2,NaN,5,NaN,2,NaN,NaN,5,4,NaN,3,NaN,NaN,1,NaN,NaN,15
Unnamed: 177,NaN,2010.5월,729450,NaN,701990,27460,NaN,554475,NaN,241695,33326,16384,348,20783,7379,29067,8030,8654,78,5291,98,7144,16,8318,711,953,456,749,105,518,22,15,2920,NaN,154066,132815,21251,NaN,NaN,NaN,112,NaN,NaN,7237,894,856,371,904,310,135,7

In [43]:
foreigner_col_list = ['Country','G.TOTAL','Foreign Visitors','Overseas Korean',
                     'ASIA','Japan','Taiwan','Hong Kong, China(including Hong Kong ID.)','Thailand',
                     'Philippines','Vietnam','China P. R.','(China)','(China-Korean)']

In [44]:
foreigner = foreigner[foreigner_col_list]

In [45]:
# foreigner.head()

In [46]:
foreigner.rename(columns={'Country' : 'Date'},inplace=True)

In [47]:
# foreigner.head(15)

In [48]:
foreigner.iloc[12::13,:].index

Index(['Unnamed: 185', 'Unnamed: 198', 'Unnamed: 211', 'Unnamed: 224',
       'Unnamed: 237', 'Unnamed: 250', 'Unnamed: 263', 'Unnamed: 276',
       'Unnamed: 289', 'Unnamed: 302'],
      dtype='object')

In [49]:
foreigner = foreigner.drop(index = foreigner.iloc[12::13,:].index)

In [50]:
# usd_krw['%change_usd_krw'] = usd_krw['%change_usd_krw'].str.replace('%','')
foreigner.Date = foreigner.Date.str.replace('.',' ')
foreigner.Date = foreigner.Date.str.replace('월','')
# usd_krw.Date = usd_krw.Date.str.replace('일','')
foreigner.Date = pd.to_datetime(foreigner.Date, format='%Y %m')
foreigner.head()

［ 입 국 ］,Date,G.TOTAL,Foreign Visitors,Overseas Korean,ASIA,Japan,Taiwan,"Hong Kong, China(including Hong Kong ID.)",Thailand,Philippines,Vietnam,China P. R.,(China),(China-Korean)
Unnamed: 173,2010-01-01,569453,546916,22537,433045,209184,36399,14545,17814,21214,4515,91252,77291,13961
Unnamed: 174,2010-02-01,638911,617573,21338,515506,230362,36651,23051,16683,20411,6749,140571,117125,23446
Unnamed: 175,2010-03-01,769894,742113,27781,602029,306126,27894,17924,25158,27441,7674,141457,102246,39211
Unnamed: 176,2010-04-01,730265,702568,27697,561228,239075,33518,20217,33398,26529,7152,147680,125230,22450
Unnamed: 177,2010-05-01,729450,701990,27460,554475,241695,33326,16384,20783,29067,7144,154066,132815,21251


# kospi, Kosdaq

In [51]:
ko_market_name = cosmetic_preprocesser.get_filelist('feature_data')[4]
ko_market_name

'kospi,kosdaq.xlsx'

In [52]:
ko_market = pd.read_excel("{}".format(ko_market_name), thousands=',')
ko_market.head()

,Symbol Name,코스닥,코스피
0,2010-01-04,528.09,1696.14
1,2010-01-05,533.09,1690.62
2,2010-01-06,539.12,1705.32
3,2010-01-07,535.52,1683.45
4,2010-01-08,538.96,1695.26


In [53]:
ko_market.columns = ['Date','kospi','kosdaq']
# ko_market.columns = add_feature_name(cobalt_rotterdam.columns,'cobalt_rotterdam')
# cobalt_rotterdam = cobalt_rotterdam.iloc[:,:4]
# cobalt_LME= co_lithum[['Date', 'price_co_lithum', 'change_co_lithum', '%change_co_lithum']]
ko_market.head()

,Date,kospi,kosdaq
0,2010-01-04,528.09,1696.14
1,2010-01-05,533.09,1690.62
2,2010-01-06,539.12,1705.32
3,2010-01-07,535.52,1683.45
4,2010-01-08,538.96,1695.26


In [54]:
ko_market.Date = pd.to_datetime(ko_market.Date)
ko_market.head()

,Date,kospi,kosdaq
0,2010-01-04,528.09,1696.14
1,2010-01-05,533.09,1690.62
2,2010-01-06,539.12,1705.32
3,2010-01-07,535.52,1683.45
4,2010-01-08,538.96,1695.26


# cny_krw

In [55]:
cny_krw_name = cosmetic_preprocesser.get_filelist('feature_data')[5]
cny_krw_name

'CNY_KRW 내역.csv'

In [56]:
cny_krw = pd.read_csv("{}".format(usd_krw_name), thousands=',')
cny_krw.head()

,날짜,종가,오픈,고가,저가,변동 %
0,2019년 12월 31일,1155.07,1157.97,1159.19,1153.14,-0.25%
1,2019년 12월 30일,1157.97,1160.97,1161.30,1154.89,-0.15%
2,2019년 12월 27일,1159.68,1162.15,1162.83,1158.12,-0.23%
3,2019년 12월 26일,1162.30,1161.04,1163.42,1160.34,0.13%
4,2019년 12월 25일,1160.76,1163.52,1164.09,1160.68,-0.24%


In [57]:
cny_krw.columns = ['Date','last','open','high','low','%change']
cny_krw.sort_values(by='Date', inplace=True)
cny_krw.columns = add_feature_name(cny_krw.columns,'cny_krw')
# usd_krw = usd_krw.iloc[:,:4]
# cobalt_LME= co_lithum[['Date', 'price_co_lithum', 'change_co_lithum', '%change_co_lithum']]

cny_krw.head()

,Date,last_cny_krw,open_cny_krw,high_cny_krw,low_cny_krw,%change_cny_krw
2607,2010년 01월 01일,1166.08,1166.08,1166.08,1166.08,0.00%
2606,2010년 01월 04일,1150.00,1164.07,1164.07,1149.35,-1.38%
2605,2010년 01월 05일,1140.20,1151.00,1151.00,1136.30,-0.85%
2604,2010년 01월 06일,1129.20,1140.20,1144.70,1129.20,-0.96%
2603,2010년 01월 07일,1138.20,1129.20,1138.20,1127.80,0.80%


In [58]:
cny_krw['%change_cny_krw'] = cny_krw['%change_cny_krw'].str.replace('%','')
cny_krw.Date = cny_krw.Date.str.replace('년 ','')
cny_krw.Date = cny_krw.Date.str.replace('월 ','')
cny_krw.Date = cny_krw.Date.str.replace('일','')
cny_krw.Date = pd.to_datetime(cny_krw.Date, format='%Y %m %d')
cny_krw.head()

,Date,last_cny_krw,open_cny_krw,high_cny_krw,low_cny_krw,%change_cny_krw
2607,2010-01-01,1166.08,1166.08,1166.08,1166.08,0.00
2606,2010-01-04,1150.00,1164.07,1164.07,1149.35,-1.38
2605,2010-01-05,1140.20,1151.00,1151.00,1136.30,-0.85
2604,2010-01-06,1129.20,1140.20,1144.70,1129.20,-0.96
2603,2010-01-07,1138.20,1129.20,1138.20,1127.80,0.80


# tech_embed

In [59]:
os.chdir(r'G:\공유 드라이브\Boad ADV Stock\화장품')
data_name ='event_embbed_with_tech_{}.xlsx'.format(company)

In [60]:
tech = pd.read_excel("{}".format(data_name), thousands=',',index_col=0)
tech.tail(20)

,date,rsi_14,macd,cci,adx,stoch_slowk,stoch_slowd,willr,momentum,roc,ema20,adosc,obv,event1,event2,event3,event4,event5,event6,event7,event8,event9,event10,event11,event12,event13,event14,event15,event16
2699,2019-12-02,83.824063,327.848620,268.950564,42.600528,82.939549,71.324677,-4.347826,2870.0,35.301353,8587.781911,5.057596e+06,291664470.0,-0.084158,-0.029610,-0.217771,-0.144692,-0.089456,-0.012022,-0.068799,-0.166941,0.170098,-0.031523,-0.323160,-0.133932,-0.234644,-0.155168,-0.206512,-0.296408
2700,2019-12-03,73.005918,432.963668,162.067247,44.784530,91.255144,81.628116,-20.289855,2400.0,29.813665,8765.136014,5.665544e+06,288570694.0,-0.008743,-0.000422,-0.135053,-0.134801,-0.182643,-0.012319,-0.056640,-0.117541,0.195429,0.006681,-0.285032,-0.095762,-0.212366,-0.137483,-0.177055,-0.205201
2701,2019-12-04,60.222569,443.376156,105.307467,44.806736,73.489576,82.561423,-50.389610,1660.0,20.880503,8845.599251,4.077382e+06,283014960.0,0.095340,0.175639,0.025462,0.185574,0.208058,0.260776,0.170650,0.053386,0.249720,0.124064,-0.058619,0.085049,0.008296,0.077633,0.045507,-0.080273
2702,2019-12-05,58.900331,438.504161,62.205330,44.827356,44.402156,69.715625,-52.987013,1710.0,21.923077,8908.875513,2.067581e+06,278441977.0,-0.117394,-0.061963,-0.241144,-0.106161,-0.161287,0.003598,-0.139283,-0.207952,0.076325,-0.115762,-0.332705,-0.150436,-0.274039,-0.198244,-0.260431,-0.295145
2703,2019-12-06,51.369663,380.231182,7.794558,43.502666,20.049828,45.980520,-69.090909,1090.0,13.974359,8907.077845,7.561635e+05,276358948.0,-0.057920,-0.008437,-0.183835,-0.109778,-0.202130,0.027132,-0.115027,-0.156296,0.177737,-0.043664,-0.303388,-0.108416,-0.241935,-0.161320,-0.237960,-0.265645
2704,2019-12-09,49.185063,314.288180,-11.170032,42.031440,5.278422,23.243468,-74.285714,580.0,7.151665,8886.403765,4.473990e+04,275765619.0,0.026494,0.078952,-0.094321,0.080987,-0.084311,0.143429,-0.037826,-0.067752,0.202816,0.001475,-0.205490,0.002310,-0.149441,-0.083377,-0.143177,-0.119154
2705,2019-12-10,50.433596,267.816732,-11.234833,40.678483,4.623501,9.983917,-71.428571,700.0,8.641975,8878.174835,-3.109638e+05,276717897.0,-0.025469,-0.002219,-0.170802,-0.093893,-0.165069,0.006277,-0.108342,-0.151626,0.187310,-0.033855,-0.292801,-0.099795,-0.226550,-0.150836,-0.239606,-0.221034
2706,2019-12-11,55.944289,269.836917,18.687723,40.158009,16.331617,8.744513,-57.922078,1280.0,15.920398,8920.253422,-3.782626e+05,278680099.0,-0.006233,0.031587,-0.117338,-0.075987,-0.048878,0.065682,-0.014473,-0.080132,0.130017,-0.010088,-0.223471,-0.067188,-0.163203,-0.078767,-0.139443,-0.156331
2707,2019-12-12,56.146236,269.940059,14.375064,39.674712,41.341709,20.765609,-60.714286,1330.0,16.604245,8960.229286,-3.912442e+05,279552963.0,-0.110238,-0.095198,-0.253696,-0.187464,-0.297964,-0.080638,-0.198345,-0.220411,0.076799,-0.119848,-0.368926,-0.182329,-0.316604,-0.221881,-0.326762,-0.285427
2708,2019-12-13,68.337714,391.389074,127.936053,41.144509,61.133008,39.602111,-28.639618,500.0,4.807692,9144.969354,-5.127649e+05,290524324.0,-0.111597,-0.094161,-0.234815,-0.182531,-0.269624,-0.050569,-0.168417,-0.217025,0.082740,-0.086420,-0.366997,-0.186729,-0.312232,-0.211467,-0.294873,-0.292932


In [61]:
tech.columns

Index(['date', 'rsi_14', 'macd', 'cci', 'adx', 'stoch_slowk', 'stoch_slowd',
       'willr', 'momentum', 'roc', 'ema20', 'adosc', 'obv', 'event1', 'event2',
       'event3', 'event4', 'event5', 'event6', 'event7', 'event8', 'event9',
       'event10', 'event11', 'event12', 'event13', 'event14', 'event15',
       'event16'],
      dtype='object')

In [62]:
tech.columns = ['Date', 'rsi_14', 'macd', 'cci', 'adx', 'stoch_slowk', 'stoch_slowd',
       'willr', 'momentum', 'roc', 'ema20', 'adosc', 'obv', 'event1', 'event2',
       'event3', 'event4', 'event5', 'event6', 'event7', 'event8', 'event9',
       'event10', 'event11', 'event12', 'event13', 'event14', 'event15',
       'event16']

# ohlcv

In [63]:
os.chdir(r'G:\공유 드라이브\Boad ADV Stock\화장품\tech_data')

tech_data = pd.read_excel('{}_tech.xlsx'.format(company))
tech_data = tech_data[5:]
tech_data.columns = ['Date','open','high','low','close','volumne']
tech_data['profit_rate'] = (tech_data['close'] - tech_data['open'])/tech_data['open']*100
# tech_data.profit_rate = tech_data.profit_rate.shift(-1)
tech_data.Date = pd.to_datetime(tech_data.Date)

In [64]:
tech_data.head()

,Date,open,high,low,close,volumne,profit_rate
5,2009-01-02,NaN,NaN,NaN,NaN,NaN,NaN
6,2009-01-05,NaN,NaN,NaN,NaN,NaN,NaN
7,2009-01-06,NaN,NaN,NaN,NaN,NaN,NaN
8,2009-01-07,NaN,NaN,NaN,NaN,NaN,NaN
9,2009-01-08,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
profit_rate = tech_data[['Date','profit_rate']]

# concat

In [66]:
data.drop('profit_rate',axis=1,inplace=True)

In [67]:
df_list = [data, kospi_vol, korean_bond, usd_krw, foreigner, ko_market, cny_krw, tech, profit_rate]

In [68]:
for list_data in df_list:
    list_data.loc[:,'Date'] = pd.to_datetime(list_data.loc[:,'Date'])
#     list_data.Date = pd.to_datetime(list_data.Date)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [69]:
for feature_data in df_list[1:]:
    data = pd.merge(data,feature_data, on ='Date',how='left')

In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2461 entries, 0 to 2460
Data columns (total 76 columns):
Date                                         2461 non-null datetime64[ns]
asset(1000)                                  1908 non-null float64
debt(1000)                                   1908 non-null float64
sales(1000)                                  2095 non-null float64
adjusted_price                               2364 non-null float64
volumne(mean_5d)(week)                       2364 non-null float64
volumne(week)                                2364 non-null float64
profit(1000)                                 2095 non-null float64
term_profit(1000)                            2095 non-null float64
gross_margin(1000)                           2095 non-null float64
adjusted_BPS                                 1908 non-null float64
adjusted_EPS                                 2095 non-null float64
last_kospi_vol                               1570 non-null float64
open_kospi_vol 

In [71]:
data.iloc[:,2:] = data.iloc[:,2:].astype('float64')

In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2461 entries, 0 to 2460
Data columns (total 76 columns):
Date                                         2461 non-null datetime64[ns]
asset(1000)                                  1908 non-null float64
debt(1000)                                   1908 non-null float64
sales(1000)                                  2095 non-null float64
adjusted_price                               2364 non-null float64
volumne(mean_5d)(week)                       2364 non-null float64
volumne(week)                                2364 non-null float64
profit(1000)                                 2095 non-null float64
term_profit(1000)                            2095 non-null float64
gross_margin(1000)                           2095 non-null float64
adjusted_BPS                                 1908 non-null float64
adjusted_EPS                                 2095 non-null float64
last_kospi_vol                               1570 non-null float64
open_kospi_vol 

In [73]:

data['rate_rolling_3'] = data.profit_rate.rolling(window=3).mean()
data['rate_rolling_5'] = data.profit_rate.rolling(window=5).mean()
data['rate_rolling_10'] = data.profit_rate.rolling(window=10).mean()
data['rate_rolling_30'] = data.profit_rate.rolling(window=30).mean()
data['rate_rolling_120'] = data.profit_rate.rolling(window=120).mean()
data['rate_rolling_150'] = data.profit_rate.rolling(window=150).mean()

data['profit_rate'] = data.profit_rate.shift(periods=-1)

In [74]:
data.head()

,Date,asset(1000),debt(1000),sales(1000),adjusted_price,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),adjusted_BPS,adjusted_EPS,last_kospi_vol,open_kospi_vol,high_kospi_vol,low_kospi_vol,%change_kospi_vol,last_korean_bond,open_korean_bond,high_korean_bond,low_korean_bond,%change_korean_bond,last_usd_krw,open_usd_krw,high_usd_krw,low_usd_krw,%change_usd_krw,G.TOTAL,Foreign Visitors,Overseas Korean,ASIA,Japan,Taiwan,"Hong Kong, China(including Hong Kong ID.)",Thailand,Philippines,Vietnam,China P. R.,(China),(China-Korean),kospi,kosdaq,last_cny_krw,open_cny_krw,high_cny_krw,low_cny_krw,%change_cny_krw,rsi_14,macd,cci,adx,stoch_slowk,stoch_slowd,willr,momentum,roc,ema20,adosc,obv,event1,event2,event3,event4,event5,event6,event7,event8,event9,event10,event11,event12,event13,event14,event15,event16,profit_rate,rate_rolling_3,rate_rolling_5,rate_rolling_10,rate_rolling_30,rate_rolling_120,rate_rolling_150
0,2010-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.840,4.840,4.840,4.840,-1.43,1120.50,1124.70,1125.4,1115.40,-0.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,540.40,1694.12,1120.50,1124.70,1125.4,1115.40,-0.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.033291,0.043504,-0.124855,-0.119890,-0.196916,0.061998,-0.053666,-0.108402,0.261804,0.051315,-0.258031,-0.042846,-0.199007,-0.104931,-0.199364,-0.204308,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.860,4.860,4.860,4.860,0.41,1120.75,1119.70,1127.3,1117.30,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,538.11,1698.64,1120.75,1119.70,1127.3,1117.30,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.027914,-0.007826,-0.179177,-0.156146,-0.212439,-0.073891,-0.144388,-0.162723,0.193392,-0.037016,-0.310264,-0.102571,-0.239889,-0.151257,-0.270027,-0.231637,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.820,4.820,4.820,4.820,-0.82,1123.00,1120.75,1130.6,1118.80,0.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,534.46,1671.41,1123.00,1120.75,1130.6,1118.80,0.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.069575,0.095009,-0.095350,-0.004076,-0.142616,0.119064,-0.018598,-0.036894,0.281866,0.079148,-0.211487,0.028184,-0.141443,-0.042114,-0.182737,-0.112766,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.775,4.775,4.775,4.775,-0.93,1123.00,1122.50,1123.9,1118.40,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,542.33,1685.77,1123.00,1122.50,1123.9,1118.40,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.201723,0.213559,0.042572,0.069579,-0.035080,0.218252,0.090860,0.070117,0.394639,0.184853,-0.075081,0.127361,-0.021977,0.065252,-0.045681,-0.007413,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.775,4.775,4.775,4.775,0.00,1123.50,1123.00,1125.3,1120.86,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,549.97,1701.80,1123.50,1123.00,1125.3,1120.86,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.089114,0.072665,-0.070127,-0.169503,-0.102870,-0.026320,-0.005249,-0.066595,0.341046,0.068563,-0.214670,-0.013746,-0.175129,-0.041270,-0.121564,-0.142835,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
os.chdir(r'G:\공유 드라이브\Boad ADV Stock\화장품')
data.to_csv("data_200_{}.csv".format(company), encoding='utf-8')